# In which the author attempts to justify his aesthetic vision of code and tests thereof

Ok, at this point, Harry is heading off criticism and concerns about the plodding pace of the unit tests introduced in Chapter 3.  He implies that if you have such feedback, you're far too naive about writing maintainable code.  He suggests building discipline in forcing yourself to follow the model of making *really* incremental changes whenever you apply TDD; the idea is to minimize the amount of work you have to re-do whenever your code fails.  If you break it up into small enough pieces, and test each separately, addressing each (inevitable) failure becomes a lot simpler.

Ok, then we resume adding stuff to `functional_tests.py` to address the issue encountered at the end of the last chapter where there was just a placeholder print statement reminding us that we hadn't done that yet.

In [3]:
%%writefile functional_tests.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(1)

        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertTrue(
            any(row.text == '1: Buy peacock feathers' for row in rows)
        )

        # There is still a text box inviting her to add another item.
        # Shee enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        self.fail('Finish the test!')

        # The page updates again, and now shows both items on her list
        # ...

if __name__ == '__main__':
    unittest.main(warnings='ignore')


Overwriting functional_tests.py


Now, I'm just a humble passive-aggressive misanthrope with a bum knee and an array of interesting sub-clinical manifestations of various other personality disorders, but to me that looks a lot *less* incremental than was preached so far.  So we'll see how he plays it.

First, we summarize the main effects of the above:

1. It introduces a few major functions from Selenium's `selenium.browser` object: `find_element_by_tag_name`, and so forth.
2. We point out that the `selenium.browser.send_keys` is the main way to simulate user input to Selenium
3. To use `send_keys`, though, we also have to import `selenium.webdriver.common.keys.Keys`, to access all the major QWERTY keyboard keys that you may need
4. The page should refresh after the `send_keys(Keys.ENTER)` command (line 38), so that `time.sleep` call is meant to give the server time to catch up before testing the assertions below; apparently this is called an "*explicit wait*"

Then, fire up the dev server and run the updated functional tests:

In [4]:
%run functional_tests

E
ERROR: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests.py", line 21, in test_can_start_a_list_and_retrieve_it_later
    header_text = self.browser.find_element_by_tag_name('h1').text
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 530, in find_element_by_tag_name
    return self.find_element(by=By.TAG_NAME, value=name)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 978, in find_element
    'value': value})['value']
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\

SystemExit: True

Ok, so it can't find an `h1` tag in the loaded HTML.  But at this point, he recommends another commit, to store the changes to the `functional_tests.py` file.


# The "Don't Test Constants" Rule, and Templates to the Rescue

He notes that the unit tests currently explicitly test for HTML tags as explicit text, which isn't how you'd do any of this if you're an actual web dev; that was just a basic way to show off that functionality before making the example a little more complex.  One TDD mantra is to skip testing constants; you wouldn't want to write `assert thing == blah` to test as trivial a line in your production code as "`thing = blah`".  And apparently that level of granularity applies analogously to HTML tags:

"*Unit tests are really about testing logic, flow control, and configuration.  Making assertions about exactly what sequence of characters we have in our HTML strings isn't doing that.*"

Furthermore, web devs don't really format HTML by writing raw tags into strings themselves; there are plenty of templates and frameworks that can do a lot of that boilerplate work for you.  Obviously, some are built into Django, so he suggests refactoring our pre-existing calls to add tags as text to instead rely on some templates.


## Refactoring to Use a Template

We define *refactoring* as changing code to achieve the same functionality as before, and specify that you should execute it on its own, rather than simultaneously refactoring *and* adding more functionality within a single commit.  Finally, you want to set up tests to try to be *sure* about the whole "same functionality" part.

We start by making a file to hold our templates:

In [28]:
%%writefile lists/templates/home.html
<html>
    <title>To-Do lists</title>
</html>

Overwriting lists/templates/home.html


Then, apparently, we also need to get rid of the explicit references to `HttpResponse` objects in our `views` file, and instead call the `django.shortcuts.render` function.  This will have Django look for folders named like "`templates`", and return the specific HTML file you request:

In [26]:
%%writefile lists/views.py
from django.shortcuts import render

def home_page(request):
    return render(request, 'home.html')


Overwriting lists/views.py


In [33]:
# %run manage test
!python manage.py test

Creating test database for alias 'default'...

..
----------------------------------------------------------------------
Ran 2 tests in 0.019s

OK



System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


### Divergent Results

Ok, at this point, my test is returning "`OK`", like everything's passing, but in the text Harry says we should see instead an

`ERROR: test_home_page_returns_correct_html (lists.tests.HomePageTest)`

where the traceback says at the bottom:

`django.template.base.TemplateDoesNotExist: home.html`

Which then prompts us to explicitly register our `lists` subdir with the `superlists/settings.py` file; specifically, in the `INSTALLED_APPS` list.

In [30]:
import inspect
from superlists import settings

for name, element in inspect.getmembers(settings):
    if name == 'INSTALLED_APPS':
        print('\n'.join(element))

django.contrib.admin
django.contrib.auth
django.contrib.contenttypes
django.contrib.sessions
django.contrib.messages
django.contrib.staticfiles


Harry says that we should have to manually add `'lists'` to that list.  Yet my test says it's passing right now, as-is.  Maybe it's a versioning difference with Django or something?

...Well, I just double-checked, and my conda env for this book is utilizing Django version `1.11.25`, and the Prereqs portion of the book just specifies we should be using Django 1.11, so I don't know if that's really a credible explanation for the difference.  Just move on, and see if other changes emerge that would allow me to troubleshoot better...

In the text, once Harry registers `lists` with `INSTALLED_APPS` in `superlists/settings.py`, he reports another error about the assertion that the home page `endswith('</html>')`, but I don't see that either.  In this case, though, he notes that you might not *always* see the error; apparently it has to do with a missing newline inserted at the end of the template.  My approach didn't add that, so I guess it's still "`OK`".

...Note: subsequently, after making the changes below, I *did* start to see an error that led me to manually add `lists` to `superlists/settings.py`.  Once I had done so, I *did* also hit the `AssertionError` that caused me to have to add the `strip()` call to the `lists/tests.py` file's `test_home_page_returns_correct_html` func.  Weird; I have no idea what caused the initial discrepancy, nor the subsequent change to become concordant with Harry's declared outputs...

Anyways, that simple replacement of the `HttpResponse("<text>")` element with the `render(template_file)` kind of approach constitutes the entirety of our "refactor" at this point.  So we're moving back to focusing on testing.  But Harry emphasizes that, in keeping with the spirit of refactor of the raw HTML input with a template file instead, we want to change what we're testing to look not for specific tags but more general ways to ensure that a response is returning the right template file.


# The Django Test Client

Harry says that the [Django Test Client](https://docs.djangoproject.com/en/1.11/topics/testing/tools/#the-test-client) is meant to intelligently detect which templates are used to generate a response.

In [24]:
%%writefile lists/tests.py
from django.urls import resolve
from django.test import TestCase
from django.http import HttpRequest

from lists.views import home_page

class HomePageTest(TestCase):

    def test_root_url_resolves_to_home_page_view(self):
        found = resolve('/')
        self.assertEqual(found.func, home_page)

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')

        html = response.content.decode('utf8')
        self.assertTrue(html.startswith('<html>'))
        self.assertIn('<title>To-Do lists</title>', html)
        self.assertTrue(html.endswith('</html>'))

        self.assertTemplateUsed(response, 'home.html')

Overwriting lists/tests.py


SO the only changes above are in the `test_home_page_returns_correct_html` func, with the removal of lines saying to generate the response via

~~~python
request = HttpRequest()
response = home_page(request)
~~~

and instead to replace them with that single call to

~~~python
response = self.client.get('/')
~~~

and finally, the addition of the final line in that func:

~~~python
self.assertTemplateUsed(response, 'home.html')
~~~

Which sounds like the more intelligent comparison method mentioned above for ensuring that the view returned comes from a certain template file.

In [34]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


..
----------------------------------------------------------------------
Ran 2 tests in 0.020s

OK


Harry then says that if you have any doubts about the effectiveness of a new test, you should purposely cause it to fail; in this case, just substitute "`wrong.html`" for "`home.html`" in the final line of the `lists/tests.py` file, and it'll tell you that a different template was used.

This new change allows you to remove a lot of the other assertions in the function we've just modified, as well as to get rid of the original unit test func:

In [36]:
%%writefile lists/tests.py
from django.test import TestCase

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')


Overwriting lists/tests.py


And ensure that it gives the same result:

In [37]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.
----------------------------------------------------------------------
Ran 1 test in 0.023s

OK


Then we commit again:

~~~bash
git status
git add lists/templates/.
git diff --staged
git commit -m "Refactor home page view to use a template"

# Adding to the Front Page

He says the functional test should still be failing (it's yielding the `NoSuchElementException`, unable to locate an `h1` tag).  Harry says that at this point it's fine to add something within an `h1` (header) tag within a `body` tag to our template, without necessarily needing to write new unit tests to detect those.

In [40]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
    </body>
</html>

Overwriting lists/templates/home.html


In [42]:
# !python functional_tests.py

Ok, at this point, he has us make multiple incremental changes to the template file, then re-run the functional test file.  Each time, however, it outputs a full traceback and a new error, because we've overlooked adding other elements that the test is looking for.  You can check them in the `functional_tests.py` file itself, but at this point, rather than cluttering up the notebook with multiple such iterations of the functional tests' output, I'll just make all of the modifications to the template and check the result.  We need to add:

1. Text within `h1` tags (done)
2. A placeholder with an `id` of "`id_new_item`"
3. A placeholder with `id` of "`id_list_table`"

To the template, but then he also notes that there's an issue with the functional test itself; specifically (in my current text), in line 43, the line checking for the text of "`1: Buy peacock feathers`" within `any` row of the `id_list_table`.  Specifically, since that table is just a placeholder at this point, this test fails and it returns a not-very-informative failure message of "`False is not true`".

In [43]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <input id="id_new_item" placeholder="Enter a to-do item" />
        <table id="id_list_table">
        </table>
    </body>
</html>

Overwriting lists/templates/home.html


In [45]:
%%writefile functional_tests.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(1)

        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertTrue(
            any(row.text == '1: Buy peacock feathers' for row in rows),
            "New to-do item did not appear in table"
        )

        # There is still a text box inviting her to add another item.
        # Shee enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        self.fail('Finish the test!')

        # The page updates again, and now shows both items on her list
        # ...

if __name__ == '__main__':
    unittest.main(warnings='ignore')


Overwriting functional_tests.py


Then, repeat the functional test:

In [47]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 44, in test_can_start_a_list_and_retrieve_it_later
    "New to-do item did not appear in table"
AssertionError: False is not true : New to-do item did not appear in table

----------------------------------------------------------------------
Ran 1 test in 7.739s

FAILED (failures=1)


Ok, so we've gotten as far as executing line #44 (#45 in the above cell because we have the first line to write the cell to file, which doesn't appear in the resulting file), and are matched up with the text.

There's a flowchart diagram (that I still need to learn how to recreate in matplotlib) summarizing the overall idea in TDD:

1. Write a test
2. Run the test
    * If it fails (expected since you haven't written prod code yet), go to #3
    * If it passes, go to #4
3. Write minimal dev/prod code
    * Go to #2
4. Does it need refactoring?
    * If yes, go to #3
    * If no, go to #1

Clearly, it's more elegant as a schematic.

But a good point that's brought up is that functional tests (again, as attempts to capture the "story" of how an end user might interact with your app) are more complex, so you can approach them a bit like apps of their own, and still apply the above cycle.  Basically, start with the functional test, and for each chunk of it that you want to get past, add new unit tests and dev/prod/app code to accomplish that, then move down to the next function in your functional test.

Commit this chapter to the repo, and move on.